In [ ]:
from collections import Counter
import re
import math
import pandas as pd
import random
import numpy as np

from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import csv,sys,time

# ntlk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist

In [ ]:
# Creating list of all Career
CareerTypes = ["Machine Learning"
               ,"Artificial Intelligence"
               ,"Deep Learning"
               ,"Android Developement"
               ,"Web Development"
               ,"Database Management"
#                ,"Consultancy"
               ,"Finance"
#                ,"Neural Network"
#                ,"Convolutions"
#                ,"Data Science"
#                ,"Business Analyst"
#                ,"IOT"
               ,"Programming"
#                ,"Designing"
#                ,"Mechanical Engineering"
#                ,"Electrical Engineering"
#                ,"Civil Engineering"
#                ,"Chemical Engineering"
               ,"Software Developer"
               ,"Game Developer"
               ,"Backend Developer"
               ,"Frontend Developer"
              ]

# Creating list of all Skills
SkillTypes = ["R",
              "C++",
              "C",
              "Python",
              "Java",
              "R",
              "Tensorflow",
              "Keraz",
#               "RNN",
#               "CNN",
#               "SVM",
#               "Unity",
              "Android Studio",
              "Photoshop",
              "Arduino",
#               "Rasberry",
#               "FPGA",
#               "Chemical",
#               "Economics",
#               "Accounting",
#               "Postgres",
              "React",
              "PhP",
              "HTML",
              "CSS",
              "NodeJs",
#               "Debating",
#               "Tableau",
              "Analog",
#               "NgSpice",
#               "Neural Network"
              ]


In [ ]:
def sanitize_input(data):
    replace = {
        ord('\f') : ' ', 
        ord('\t') : ' ',
        ord('\n') : ' ',
        ord('\r') : None
    }
    return data.translate(replace)

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def tokenize_content(content):
    stop_words = set(stopwords.words('english') + list(punctuation))
    words = word_tokenize(content.lower())
    return [word for word in words if word not in stop_words]

def word_freq(content):
    filtered_words = tokenize_content(content)
    word_freq = FreqDist(filtered_words)
    return word_freq

In [ ]:
All_Resume = {}
TotalResumes = 0

#### Settings #### 
options = webdriver.ChromeOptions()
prefs={"profile.managed_default_content_settings.images": 2,'disk-cache-size': 4096 }
options.add_experimental_option('prefs', prefs)
options.headless = True
#options.add_argument("user-data-dir=/Users/manthan/Library/Application Support/Google/Chrome")
#chrome_options.add_argument("--incognito")
options.add_argument('disable-infobars')
driver = webdriver.Chrome(executable_path='chromedriver', options=options)

def frontpage2(arr, link1):
    driver.get(link1)
    about_page = driver.page_source    
    abt_soup = bs(about_page, "html.parser")
    
    ## Find Text
    heading = abt_soup.find('h3').text
    desc = abt_soup.find("div", attrs = {'class':'jobsearch-jobDescriptionText'}).text
#     loc=abt_soup.find('span',{'class':'jobsearch-JobMetadataHeader-iconLabel'})
#     exp=abt_soup.find('span',{'class':'jobsearch-JobMetadataHeader-iconLabel'})
#     company = abt_soup.find("div", attrs = {'class':'icl-u-lg-mr--sm icl-u-xs-mr--xs'})    
    JobDescriptionText = sanitize_input(heading+" "+desc)
    JobDescriptionText = " ".join(tokenize_content(JobDescriptionText))
    arr.append(JobDescriptionText)
    
def frontpage(All_Resume, career,ResumesUrl):
    JD = []
    driver.get(ResumesUrl)
    page_source=driver.page_source
    soup = bs(page_source, "html.parser")
    for link in soup.findAll("a", {"class": "jobtitle"}):
        dp_link = link.get('href')
        if dp_link.split(":")[0]=='https':
            frontpage2(JD,dp_link)
        else:
            frontpage2(JD,"https://www.indeed.co.in" + dp_link)
    All_Resume[career] = JD

for career in CareerTypes:
    for i in range(5):
        frontpage(All_Resume, career,'https://www.indeed.co.in/jobs?q='+career+'&start='+str(i)+'0')
        TotalResumes += len(All_Resume[career])

driver.quit()

In [ ]:
def termFrequency (skill, skill_resume):
    i = 0.0;
    for c in skill_resume:
        cmt = c.count(skill)
        if(cmt>0):
            i+=1.0
    total_resume = len(skill_resume)
    try:
        termFrequency = i/total_resume
    except:
        termFrequency = 0
    return termFrequency
def SkillCountInSkill(job, job_resumes):
    count = 0
    for x in job_resumes:
        if(x.count(job)>0):
            count+=1
    return count
def idf (job, all_resumes):
    N = TotalResumes
    nw = 1
    for skill_resume in all_resumes:
        nw += SkillCountInSkill(job,skill_resume)
    idf = math.log(N/(nw+1))
    return idf

In [ ]:
## Calculates P(i/j)
def relevance(i,j):
    return idf(i.lower(),All_Resume)*termFrequency(i.lower(),All_Resume[j])

In [ ]:
## Calculating Career Skill Matrix:
## find relevence of a page on the basis of query 
CareerSkillMatrix = []
for j in CareerTypes:
    y = [relevance(i,j) for i in SkillTypes ]
    CareerSkillMatrix.append(y)
print(CareerSkillMatrix)

In [ ]:
def SkillCount(x,text):
    res = text.count(x.lower())
    return (1 if res > 0 else 0)
def BoolSkillMatrix(text):
    BoolSkillMatrixJD = [SkillCount(x,text) for x in SkillTypes]
    return np.array([BoolSkillMatrixJD])

In [ ]:
## Calculating Skill Skill Matrix:
s = len(SkillTypes)
SkillMatrix_JD = np.zeros((s,s))
for career in CareerTypes:
    for jd in All_Resume[career]:
        X = BoolSkillMatrix(jd)
        X = X.T.dot(X)
        SkillMatrix_JD += X
print(SkillMatrix_JD)
maxE = SkillMatrix_JD.max(axis=0)
SkillMatrix_JD = (SkillMatrix_JD.T)/maxE
SkillMatrix_JD = SkillMatrix_JD.T
print(SkillMatrix_JD)

In [265]:
## Finding The User Skill Matrix

In [266]:
def getUserSkillMatrix(CV):
    UserSkillArr = BoolSkillMatrix(CV)
    x = np.argwhere(UserSkillArr==1)
    SkillMatrix = SkillMatrix_JD[x[:,1],:]
    return SkillMatrix.max(axis=0)
def getJobDescriptionSkillMatrix(JD):
    BoolSkillMatrixJD = BoolSkillMatrix(JD)
    return BoolSkillMatrixJD
def SimilaritySkillMatrix(M1,M2):
    return (M1.dot(M2.T))/math.sqrt(( np.sum(M1**2) * np.sum(M2**2) ))

In [267]:
JD = ("Deep Learning Engineer Deep Learning Engineer Bengaluru, Karnataka To gain employment with a company that offers me consistently positive atmosphere to learn new technologies and implement them for the betterment of the business."
     "Work Experience Deep Learning Engineer DronaMaps Pvt Ltd June 2018 to May 2019 Gurugram Currently, I am working on aerial images which are taken from drones for processing those images in useful result by applying deep learning and image processing techniques."
     "Industrial Projects • Building footprint detection from aerial images of drone - Use CNN (U-NET) to apply semantic segmentation on aerial images to find the footprint of building. • Road detection from aerial images of drone - Use CNN (U-NET) to apply semantic "
     "gmentation on aerial images to find the road. • Building type classification from aerial images of drone - Classify building in four different class in aerial drone images using CNN (Dense Net) • Crop classification based on drone aerial images - To classify crops"
     "drone aerial images using CNN • Plant stress detection - Apply Kernel method on aerial images for identification of plant stress. - Use Neural Network for finding plant stress from aerial image Capturing from drone and creating dataset. • Plant population count"
     "ed Convolutions Neural Network (CNN) to counting plant from aerial images of drone. • Depth estimation from RGB images using Deep Learning for Indoor Mapping - RGBD Salient Object Detection data set used to train CNN model for esti- mation depth map of RGB images."
     "Aerial image classification   Data set is created in hdf5 format, which contains usable images and un- usable images for data processing. CNN is used for classify usable images • 3D Reconstruction using Aerial Drone Images - Data acquisition using drone and process "
     "on Open Drone Maps. • Point Cloud Classification - In the process of 3D reconstruction point cloud is generated. Deep Learning model (Point-Net) is used on point cloud for point cloud classification. NEURAL (Less than 1 year), NEURAL NETWORK (Less than 1 year), "
     "DEEP LEARNING ss than 1 year), ESTIMATION (Less than 1 year), SEGMENTATION (Less than 1 year) https://github.com/Shahzadnit Additional Information Technical Proficiency Computer Vision: Classification, Regression, Semantic Segmentation, Depth estimation "
     "ep Learning Convolutional Neural Network, Auto-encoder, GAN Machine Learning Neural Network, SVM Mathematics Optimization, Backpropogation of Neural Network General Programming C, C++, Python, OpenCV-python, PyTorch, MATLAB, LaTeX")
JD = RefineText(JD).lower()

In [268]:
## Finding Similarity Score between User Skill Matrix and Job Description Skill Matrix## Finding the Job Description Skill Matrix
UserSkillMatrix = getUserSkillMatrix(JD)
JDSkillMatrixBool = getJobDescriptionSkillMatrix(JD)
RelevenceOfUserSkill = JDSkillMatrixBool*UserSkillMatrix
Score = SimilaritySkillMatrix(RelevenceOfUserSkill,JDSkillMatrixBool)[0][0]
print(Score)

1.0
